In [138]:
import pandas as pd
import numpy as np
import collections
import gensim 
from gensim.models import word2vec, phrases
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_numeric,\
                    strip_non_alphanum, strip_multiple_whitespaces, strip_short
from textblob import TextBlob, Word

import re
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
%matplotlib inline


from strsimpy.cosine import Cosine

In [229]:
recipe = pd.read_csv('core-data_recipe.csv')
recipe = recipe.drop(columns=['image_url'])
recipe.head()

,recipe_id,recipe_name,ingredients,cooking_directions,nutritions
0,240488,"Pork Loin, Apples, and Sauerkraut",sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."
1,218939,Foolproof Rosemary Chicken Wings,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'..."
2,87211,Chicken Pesto Paninis,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
3,245714,Potato Bacon Pizza,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
4,218545,Latin-Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves^diced ...,{'directions': u'Prep\n10 m\nCook\n8 h 15 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."


In [230]:
df = recipe[['recipe_id','recipe_name', 'ingredients']]
df.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

In [231]:
df.head()

,recipe_id,recipe_name,ingredients
0,240488,Pork Loin Apples and Sauerkraut,sauerkraut drained Granny Smith apples sliced ...
1,218939,Foolproof Rosemary Chicken Wings,chicken wings sprigs rosemary head garlic oliv...
2,87211,Chicken Pesto Paninis,focaccia bread quartered prepared basil pesto ...
3,245714,Potato Bacon Pizza,red potatoes strips bacon Sauce heavy whippin...
4,218545,Latin Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves diced ...


In [232]:
df['details'] = df['recipe_name'].str.lower() + ', '+ df['ingredients'].str.lower()

C:\Users\wen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [233]:
df.head()

,recipe_id,recipe_name,ingredients,details
0,240488,Pork Loin Apples and Sauerkraut,sauerkraut drained Granny Smith apples sliced ...,"pork loin apples and sauerkraut, sauerkraut ..."
1,218939,Foolproof Rosemary Chicken Wings,chicken wings sprigs rosemary head garlic oliv...,"foolproof rosemary chicken wings, chicken wing..."
2,87211,Chicken Pesto Paninis,focaccia bread quartered prepared basil pesto ...,"chicken pesto paninis, focaccia bread quartere..."
3,245714,Potato Bacon Pizza,red potatoes strips bacon Sauce heavy whippin...,"potato bacon pizza, red potatoes strips bacon ..."
4,218545,Latin Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves diced ...,"latin inspired spicy cream chicken stew, skinl..."


In [234]:
details=[]
for row in range(0,len(df.index)):
    details.append(df.iloc[row,3])

In [235]:
details

['pork loin  apples  and sauerkraut, sauerkraut drained granny smith apples sliced large onion caraway seeds apple cider divided brown sugar rub  thai seasoning salt garlic powder ground black pepper boneless pork loin roast',
 'foolproof rosemary chicken wings, chicken wings sprigs rosemary head garlic olive oil lemon pepper seasoned salt',
 'chicken pesto paninis, focaccia bread quartered prepared basil pesto diced cooked chicken diced green bell pepper diced red onion shredded monterey jack cheese',
 'potato bacon pizza, red potatoes strips bacon sauce  heavy whipping cream butter minced garlic grated parmesan cheese crust  warm water      degrees f    degrees c  honey active dry yeast vegetable oil all purpose flour shredded mozzarella cheese',
 'latin inspired spicy cream chicken stew, skinless boneless chicken breast halves diced tomatoes green salsa black beans rinsed and drained pinto beans drained and rinsed kernel corn taco seasoning chopped fresh cilantro ground red chile pe

In [236]:
# input json  
#{
#    "uuid": 1234,
#    "user_id": 1234,
#    "request_id": 1,
#    "request_time": "9/28/2020",
#    "user input": {
#        "motherhood_stage": "pregnancy first trimester",
#        "ingredient_list": ["chicken", "tomato"],
#        "nutrition_focus": ["full-eval","iron"],
#        "if_vegetierian": true,
#        "ingredient_exclusion": ["onion","cilantro","cheese"]
#    }
#}

In [237]:
json_input = 'chicken tomato potato'

In [238]:
df['input'] = json_input

In [239]:
df.head()

,recipe_id,recipe_name,ingredients,details,input
0,240488,Pork Loin Apples and Sauerkraut,sauerkraut drained Granny Smith apples sliced ...,"pork loin apples and sauerkraut, sauerkraut ...",chicken tomato potato
1,218939,Foolproof Rosemary Chicken Wings,chicken wings sprigs rosemary head garlic oliv...,"foolproof rosemary chicken wings, chicken wing...",chicken tomato potato
2,87211,Chicken Pesto Paninis,focaccia bread quartered prepared basil pesto ...,"chicken pesto paninis, focaccia bread quartere...",chicken tomato potato
3,245714,Potato Bacon Pizza,red potatoes strips bacon Sauce heavy whippin...,"potato bacon pizza, red potatoes strips bacon ...",chicken tomato potato
4,218545,Latin Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves diced ...,"latin inspired spicy cream chicken stew, skinl...",chicken tomato potato


In [240]:
cosine = Cosine(2)
df["p0"] = df["input"].apply(lambda s: cosine.get_profile(s)) 
df["p1"] = df["details"].apply(lambda s: cosine.get_profile(s)) 
df["cosine_sim"] = [cosine.similarity_profiles(p0,p1) for p0,p1 in zip(df["p0"],df["p1"])]

df_2 = df.drop(["p0", "p1"], axis=1)

In [403]:
searchfor = ['chicken broth','tomato','potato']
df_3 = df_2[df_2['details'].str.contains('|'.join(searchfor))]
df_4 = df_3[df_3['details'].str.contains('chicken broth')]


In [404]:
df_4

,recipe_id,recipe_name,ingredients,details,input,cosine_sim
22,235757,Chicken Pot Pie a la Jeremy and Jasmin,butter chopped onion all purpose flour salt gr...,"chicken pot pie a la jeremy and jasmin, butter...",chicken tomato potato,0.257920
26,231459,Slow Cooker Pork with Mushrooms and Barley,garlic salt pork chops barley white mushrooms ...,"slow cooker pork with mushrooms and barley, ga...",chicken tomato potato,0.286034
28,133604,Jen s Jambalaya,margarine chopped onion chopped celery chopped...,"jen s jambalaya, margarine chopped onion chopp...",chicken tomato potato,0.311034
29,55420,The Absolute Best Potato Soup,fluid ounce can chicken broth country style g...,"the absolute best potato soup, fluid ounce ca...",chicken tomato potato,0.393867
38,241535,Easy Curry Chicken,Reynolds Slow Cooker Liner chicken broth skin...,"easy curry chicken, reynolds slow cooker line...",chicken tomato potato,0.228274
40,244979,Chef John s Peanut Curry Chicken,Spice Blend kosher salt plus more to taste gr...,"chef john s peanut curry chicken, spice blend ...",chicken tomato potato,0.304270
74,245367,Pan Roasted Chicken Breasts,boneless chicken breast halves with skin salt ...,"pan roasted chicken breasts, boneless chicken ...",chicken tomato potato,0.296658
95,11730,Herbed Chicken and Fettuccini,dry fettuccine pasta olive oil skinless bonele...,"herbed chicken and fettuccini, dry fettuccine ...",chicken tomato potato,0.304820
103,81576,Baked Chicken with Applesauce Stuffing,dry bread crumbs onion celery applesauce chick...,"baked chicken with applesauce stuffing, dry br...",chicken tomato potato,0.289227
141,88690,Jackie s Sausage Soup,links sweet Italian sausage green bell pepper ...,"jackie s sausage soup, links sweet italian sau...",chicken tomato potato,0.308303


In [255]:
df_3.shape

(15235, 6)

In [256]:
df_4.shape

(7410, 6)

In [265]:
output = df_4.nlargest(10, 'cosine_sim')

In [266]:
output

,recipe_id,recipe_name,ingredients,details,input,cosine_sim
3874,8905,Golden Vegetable Chicken,to pound whole chicken onion potatoes carro...,"golden vegetable chicken, to pound whole ch...",chicken tomato potato,0.643576
2617,241808,Chicken Stew,water chicken tenders carrots celery potatoes ...,"chicken stew , water chicken tenders carrots c...",chicken tomato potato,0.637307
32804,79383,Chunky Tomato Potato Soup,butter onions peeled cubed potatoes chopped ce...,"chunky tomato potato soup, butter onions peele...",chicken tomato potato,0.627022
43617,17900,Tomato Florentine Soup II,chicken stock tomato sauce tomato juice tomato...,"tomato florentine soup ii, chicken stock tomat...",chicken tomato potato,0.611132
494,16398,Chinese Chicken and Potato Soup,potatoes carrot turnip chopped onion garlic ch...,"chinese chicken and potato soup, potatoes carr...",chicken tomato potato,0.600751
44941,23996,Aaahh Potato Soup,Yukon Gold potatoes au gratin instant potato m...,"aaahh potato soup, yukon gold potatoes au gra...",chicken tomato potato,0.598353
9757,15737,Quick and Easy Enchiladas,corn oil onion tomato chicken chunks salt and ...,"quick and easy enchiladas, corn oil onion toma...",chicken tomato potato,0.591803
26141,23062,Quick Spaghetti Sauce,green bell pepper onion garlic tomatoes Italia...,"quick spaghetti sauce, green bell pepper onion...",chicken tomato potato,0.584048
25799,16965,Lower Fat Potato Soup,onion celery fat free chicken broth potatoes d...,"lower fat potato soup, onion celery fat free c...",chicken tomato potato,0.581954
32831,216719,Cabbage Potato and Tomato Soup,butter onion potatoes celery garlic water chic...,"cabbage potato and tomato soup, butter onion...",chicken tomato potato,0.579392


In [267]:
output.iloc[0,2]

'to   pound  whole chicken onion potatoes carrots sweet potato tomato mayonnaise cayenne pepper ground turmeric garlic powder salt and pepper to taste'

In [268]:
output.iloc[1,2]

'water chicken tenders carrots celery potatoes sweet potato peas tomato sauce bay leaves cooked rice'

In [269]:
output.iloc[2,2]

'butter onions peeled cubed potatoes chopped celery chopped carrots garlic Italian seasoning milk cornstarch tomatoes chicken broth tomato paste salt and pepper to taste'

In [270]:
output.iloc[3,2]

'chicken stock tomato sauce tomato juice tomato paste white sugar celery chopped carrots chopped spinach diced zucchini cooked pasta ground nutmeg salt and pepper to taste'

In [272]:
output.iloc[4,2]

'potatoes carrot turnip chopped onion garlic chicken leg salt and pepper to taste'

In [264]:
result = output[['recipe_id']]

In [186]:
result

,recipe_id
3874,8905
2617,241808
32804,79383
26119,20968
27879,14081
43617,17900
43144,21083
9001,15914
44941,23996
494,16398


In [368]:
nutrition = recipe[['recipe_id','nutritions']]

In [369]:
nutrition.head()

,recipe_id,nutritions
0,240488,"{u'niacin': {u'hasCompleteData': False, u'name..."
1,218939,"{u'niacin': {u'hasCompleteData': True, u'name'..."
2,87211,"{u'niacin': {u'hasCompleteData': True, u'name'..."
3,245714,"{u'niacin': {u'hasCompleteData': True, u'name'..."
4,218545,"{u'niacin': {u'hasCompleteData': False, u'name..."


In [372]:
nutrition.iloc[0,1]

"{u'niacin': {u'hasCompleteData': False, u'name': u'Niacin Equivalents', u'amount': 15.6016, u'percentDailyValue': u'120', u'displayValue': u'16', u'unit': u'mg'}, u'sugars': {u'hasCompleteData': False, u'name': u'Sugars', u'amount': 19.84146, u'percentDailyValue': u'0', u'displayValue': u'19.8', u'unit': u'g'}, u'sodium': {u'hasCompleteData': False, u'name': u'Sodium', u'amount': 2606.764, u'percentDailyValue': u'104', u'displayValue': u'2607', u'unit': u'mg'}, u'carbohydrates': {u'hasCompleteData': True, u'name': u'Carbohydrates', u'amount': 32.08176, u'percentDailyValue': u'10', u'displayValue': u'32.1', u'unit': u'g'}, u'vitaminB6': {u'hasCompleteData': False, u'name': u'Vitamin B6', u'amount': 1.328631, u'percentDailyValue': u'83', u'displayValue': u'1', u'unit': u'mg'}, u'calories': {u'hasCompleteData': True, u'name': u'Calories', u'amount': 371.7219, u'percentDailyValue': u'19', u'displayValue': u'372', u'unit': u'kcal'}, u'thiamin': {u'hasCompleteData': False, u'name': u'Thiami

In [359]:
import csv
import json

with open('test.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        single_json = row[1]
        single_json = single_json.replace("u'", "'")
        single_json = single_json.replace("'", '"')
        data = json.loads(single_json)
        print (json.dumps(data, indent=4))

{
    "Other": {
        "Text": "Telephone",
        "start": 45,
        "end": 54,
        "value": "contact information"
    }
}


In [398]:
import csv
import json

with open('nutrition_3.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        single_json = row[1]
        single_json = single_json.replace("u'", "'")
        single_json = single_json.replace("'", '"')
        single_json = single_json.replace("False", '0')
        single_json = single_json.replace("True", '1')
        data = json.loads(single_json)
        print (json.dumps(data, indent=4))


{
    "niacin": {
        "hasCompleteData": 0,
        "name": "Niacin Equivalents",
        "amount": 15.6016,
        "percentDailyValue": "120",
        "displayValue": "16",
        "unit": "mg"
    },
    "sugars": {
        "hasCompleteData": 0,
        "name": "Sugars",
        "amount": 19.84146,
        "percentDailyValue": "0",
        "displayValue": "19.8",
        "unit": "g"
    },
    "sodium": {
        "hasCompleteData": 0,
        "name": "Sodium",
        "amount": 2606.764,
        "percentDailyValue": "104",
        "displayValue": "2607",
        "unit": "mg"
    },
    "carbohydrates": {
        "hasCompleteData": 1,
        "name": "Carbohydrates",
        "amount": 32.08176,
        "percentDailyValue": "10",
        "displayValue": "32.1",
        "unit": "g"
    },
    "vitaminB6": {
        "hasCompleteData": 0,
        "name": "Vitamin B6",
        "amount": 1.328631,
        "percentDailyValue": "83",
        "displayValue": "1",
        "unit": "mg"
    

{
    "niacin": {
        "hasCompleteData": 0,
        "name": "Niacin Equivalents",
        "amount": 27.97977,
        "percentDailyValue": "215",
        "displayValue": "28",
        "unit": "mg"
    },
    "sugars": {
        "hasCompleteData": 0,
        "name": "Sugars",
        "amount": 9.964849,
        "percentDailyValue": "0",
        "displayValue": "10",
        "unit": "g"
    },
    "sodium": {
        "hasCompleteData": 0,
        "name": "Sodium",
        "amount": 546.75,
        "percentDailyValue": "22",
        "displayValue": "547",
        "unit": "mg"
    },
    "carbohydrates": {
        "hasCompleteData": 0,
        "name": "Carbohydrates",
        "amount": 62.27578,
        "percentDailyValue": "20",
        "displayValue": "62.3",
        "unit": "g"
    },
    "vitaminB6": {
        "hasCompleteData": 0,
        "name": "Vitamin B6",
        "amount": 1.42994,
        "percentDailyValue": "89",
        "displayValue": "1",
        "unit": "mg"
    },
   

JSONDecodeError: Expecting value: line 1 column 43 (char 42)

In [400]:
nutrition_df = pd.DataFrame(data)

In [402]:
nutrition_df.head(100)

,niacin,sugars,sodium,carbohydrates,vitaminB6,calories,thiamin,fat,folate,caloriesFromFat,calcium,fiber,magnesium,iron,cholesterol,protein,vitaminA,potassium,saturatedFat,vitaminC
amount,8.5162,0.454489,1021.91,26.4912,0.226918,639.127,0.480731,48.5613,55.739,437.052,347.906,1.38245,28.4721,3.03445,138.428,24.3041,1117.96,243.355,26.9271,0.723825
displayValue,9,0.5,1022,26.5,< 1,639,< 1,48.6,56,437,348,1.4,28,3,138,24.3,1118,243,26.9,< 1
hasCompleteData,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1
name,Niacin Equivalents,Sugars,Sodium,Carbohydrates,Vitamin B6,Calories,Thiamin,Fat,Folate,Calories from Fat,Calcium,Dietary Fiber,Magnesium,Iron,Cholesterol,Protein,Vitamin A - I,Potassium,Saturated Fat,Vitamin C
percentDailyValue,66,0,41,9,14,32,48,75,31,-,43,6,10,30,46,49,22,7,135,1
unit,mg,g,mg,g,mg,kcal,mg,g,mcg,kcal,mg,g,mg,mg,mg,g,I,mg,g,mg


In [386]:
data

{'niacin': {'hasCompleteData': 1,
  'name': 'Niacin Equivalents',
  'amount': 8.516196,
  'percentDailyValue': '66',
  'displayValue': '9',
  'unit': 'mg'},
 'sugars': {'hasCompleteData': 1,
  'name': 'Sugars',
  'amount': 0.4544888,
  'percentDailyValue': '0',
  'displayValue': '0.5',
  'unit': 'g'},
 'sodium': {'hasCompleteData': 1,
  'name': 'Sodium',
  'amount': 1021.906,
  'percentDailyValue': '41',
  'displayValue': '1022',
  'unit': 'mg'},
 'carbohydrates': {'hasCompleteData': 1,
  'name': 'Carbohydrates',
  'amount': 26.49121,
  'percentDailyValue': '9',
  'displayValue': '26.5',
  'unit': 'g'},
 'vitaminB6': {'hasCompleteData': 1,
  'name': 'Vitamin B6',
  'amount': 0.2269181,
  'percentDailyValue': '14',
  'displayValue': '< 1',
  'unit': 'mg'},
 'calories': {'hasCompleteData': 1,
  'name': 'Calories',
  'amount': 639.1274,
  'percentDailyValue': '32',
  'displayValue': '639',
  'unit': 'kcal'},
 'thiamin': {'hasCompleteData': 0,
  'name': 'Thiamin',
  'amount': 0.4807308,
  

In [ ]:
with open('nutrition_3.csv') as f:
    data = []
    text = f.read()
    records = [re.split(",(?![ \n]+)", x.replace('"', '"')) for x in re.split("(?<!, )\n", text)]
    for r in records[1:]:
        second_col = json.loads(r[1])
        for d in second_col:
            d['GenreId'] = d['id']
            d['Id'] = r[0]
            del d['id']
            d['GenreName'] = d['name']
            del d['name']
        data += second_col
    df = pd.DataFrame(data)
    
    
    with open('nutrition_3.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        single_json = row[1]
        single_json = single_json.replace("u'", "'")
        single_json = single_json.replace("'", '"')
        single_json = single_json.replace("False", '0')
        single_json = single_json.replace("True", '1')
        data = json.loads(single_json)

In [385]:
data2

'{\n    "niacin": {\n        "hasCompleteData": 1,\n        "name": "Niacin Equivalents",\n        "amount": 8.516196,\n        "percentDailyValue": "66",\n        "displayValue": "9",\n        "unit": "mg"\n    },\n    "sugars": {\n        "hasCompleteData": 1,\n        "name": "Sugars",\n        "amount": 0.4544888,\n        "percentDailyValue": "0",\n        "displayValue": "0.5",\n        "unit": "g"\n    },\n    "sodium": {\n        "hasCompleteData": 1,\n        "name": "Sodium",\n        "amount": 1021.906,\n        "percentDailyValue": "41",\n        "displayValue": "1022",\n        "unit": "mg"\n    },\n    "carbohydrates": {\n        "hasCompleteData": 1,\n        "name": "Carbohydrates",\n        "amount": 26.49121,\n        "percentDailyValue": "9",\n        "displayValue": "26.5",\n        "unit": "g"\n    },\n    "vitaminB6": {\n        "hasCompleteData": 1,\n        "name": "Vitamin B6",\n        "amount": 0.2269181,\n        "percentDailyValue": "14",\n        "displayV

In [367]:
nutrition_df = pd.read_json(data)

ValueError: Invalid file path or buffer object type: <class 'dict'>

In [373]:
nutrition['nutritions'] = nutrition['nutritions'].str.replace("u'", "'")

C:\Users\wen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [374]:
nutrition['nutritions'] = nutrition['nutritions'].str.replace("'", '"')

C:\Users\wen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [375]:
nutrition['nutritions'] = nutrition['nutritions'].str.replace("\n", ' ')

C:\Users\wen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [376]:
nutrition['nutritions'] = nutrition['nutritions'].str.replace("False", '0')

C:\Users\wen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [377]:
nutrition['nutritions'] = nutrition['nutritions'].str.replace("False", '0')

C:\Users\wen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [378]:
nutrition['nutritions'] = nutrition['nutritions'].str.replace("True", '1')

C:\Users\wen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [379]:
nutrition.head()

,recipe_id,nutritions
0,240488,"{""niacin"": {""hasCompleteData"": 0, ""name"": ""Nia..."
1,218939,"{""niacin"": {""hasCompleteData"": 1, ""name"": ""Nia..."
2,87211,"{""niacin"": {""hasCompleteData"": 1, ""name"": ""Nia..."
3,245714,"{""niacin"": {""hasCompleteData"": 1, ""name"": ""Nia..."
4,218545,"{""niacin"": {""hasCompleteData"": 0, ""name"": ""Nia..."


In [380]:
nutrition.iloc[0,1]

'{"niacin": {"hasCompleteData": 0, "name": "Niacin Equivalents", "amount": 15.6016, "percentDailyValue": "120", "displayValue": "16", "unit": "mg"}, "sugars": {"hasCompleteData": 0, "name": "Sugars", "amount": 19.84146, "percentDailyValue": "0", "displayValue": "19.8", "unit": "g"}, "sodium": {"hasCompleteData": 0, "name": "Sodium", "amount": 2606.764, "percentDailyValue": "104", "displayValue": "2607", "unit": "mg"}, "carbohydrates": {"hasCompleteData": 1, "name": "Carbohydrates", "amount": 32.08176, "percentDailyValue": "10", "displayValue": "32.1", "unit": "g"}, "vitaminB6": {"hasCompleteData": 0, "name": "Vitamin B6", "amount": 1.328631, "percentDailyValue": "83", "displayValue": "1", "unit": "mg"}, "calories": {"hasCompleteData": 1, "name": "Calories", "amount": 371.7219, "percentDailyValue": "19", "displayValue": "372", "unit": "kcal"}, "thiamin": {"hasCompleteData": 0, "name": "Thiamin", "amount": 0.8423121, "percentDailyValue": "84", "displayValue": "< 1", "unit": "mg"}, "fat":

In [382]:
nutrition.iloc[1,1]

'{"niacin": {"hasCompleteData": 1, "name": "Niacin Equivalents", "amount": 10.06679, "percentDailyValue": "77", "displayValue": "10", "unit": "mg"}, "sugars": {"hasCompleteData": 0, "name": "Sugars", "amount": 0.2361299, "percentDailyValue": "0", "displayValue": "0.2", "unit": "g"}, "sodium": {"hasCompleteData": 1, "name": "Sodium", "amount": 762.805, "percentDailyValue": "31", "displayValue": "763", "unit": "mg"}, "carbohydrates": {"hasCompleteData": 1, "name": "Carbohydrates", "amount": 6.071931, "percentDailyValue": "2", "displayValue": "6.1", "unit": "g"}, "vitaminB6": {"hasCompleteData": 1, "name": "Vitamin B6", "amount": 0.5538, "percentDailyValue": "35", "displayValue": "< 1", "unit": "mg"}, "calories": {"hasCompleteData": 1, "name": "Calories", "amount": 335.1655, "percentDailyValue": "17", "displayValue": "335", "unit": "kcal"}, "thiamin": {"hasCompleteData": 0, "name": "Thiamin", "amount": 0.06496, "percentDailyValue": "6", "displayValue": "< 1", "unit": "mg"}, "fat": {"hasCo

In [381]:
nutritions_df = nutrition['nutritions'].apply(json.loads)

JSONDecodeError: Expecting value: line 1 column 43 (char 42)

In [350]:
nutrition.join(nutrition['nutritions'].apply(json.loads).apply(pd.Series))

JSONDecodeError: Expecting value: line 1 column 32 (char 31)

In [351]:
from pandas.io.json import json_normalize
json_normalize(nutrition['nutritions'])

AttributeError: 'str' object has no attribute 'values'

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [290]:
arr = nutrition['nutritions'].to_numpy

In [394]:
nutrition = pd.read_csv('nutrition_4.csv', header = 0)

In [395]:
nutrition.head()

,recipe_id,nutrition
0,240488.0,"{""niacin"": {""hasCompleteData"": 0, ""name"": ""Nia..."
1,218939.0,"{""niacin"": {""hasCompleteData"": 1, ""name"": ""Nia..."
2,87211.0,"{""niacin"": {""hasCompleteData"": 1, ""name"": ""Nia..."
3,245714.0,"{""niacin"": {""hasCompleteData"": 1, ""name"": ""Nia..."
4,218545.0,"{""niacin"": {""hasCompleteData"": 0, ""name"": ""Nia..."


In [397]:
nutrition.iloc[0,1]

'{"niacin": {"hasCompleteData": 0, "name": "Niacin Equivalents", "amount": 15.6016, "percentDailyValue": "120", "displayValue": "16", "unit": "mg"}, "sugars": {"hasCompleteData": 0, "name": "Sugars", "amount": 19.84146, "percentDailyValue": "0", "displayValue": "19.8", "unit": "g"}, "sodium": {"hasCompleteData": 0, "name": "Sodium", "amount": 2606.764, "percentDailyValue": "104", "displayValue": "2607", "unit": "mg"}, "carbohydrates": {"hasCompleteData": 1, "name": "Carbohydrates", "amount": 32.08176, "percentDailyValue": "10", "displayValue": "32.1", "unit": "g"}, "vitaminB6": {"hasCompleteData": 0, "name": "Vitamin B6", "amount": 1.328631, "percentDailyValue": "83", "displayValue": "1", "unit": "mg"}, "calories": {"hasCompleteData": 1, "name": "Calories", "amount": 371.7219, "percentDailyValue": "19", "displayValue": "372", "unit": "kcal"}, "thiamin": {"hasCompleteData": 0, "name": "Thiamin", "amount": 0.8423121, "percentDailyValue": "84", "displayValue": "< 1", "unit": "mg"}, "fat":

In [396]:
nutritions_df = nutrition['nutrition'].apply(json.loads)

JSONDecodeError: Expecting value: line 1 column 43 (char 42)